In [6]:
from IPython.display import display
import numpy as np
import scipy.sparse
import scipy.linalg as la
import theano
import theano.tensor.slinalg
import theano.tensor as T

n = 4

A = np.zeros(shape=(n, n), dtype=np.float)
A += scipy.sparse.rand(n, n, density=.8)
A = np.asarray(A)


# ----------- numerical differentiation
x = 2.
h = 0.001
numerical_diff = (la.expm((x + h) * A) - la.expm(x * A)) / h

# display gradient computed with numerical differentiation
display(numerical_diff)



# ----------- automatic differentiation with theano
x = T.dscalar('x')
expA = T.slinalg.expm(x * A)
# the flattening is for more easily scan through the elements of the matrix
# (theano.grad only accepts scalar cost)
expA_flat = T.flatten(expA)

def compute_element_grad(idx, flattened_matrix):
    return T.grad(flattened_matrix[idx], wrt=x)
# `theano.scan` basically loops over the elements of the matrix, and returns the gradient of each 
g_x_flat, _ = theano.scan(
    fn=compute_element_grad,
    sequences=T.arange(expA_flat.shape[0]),
    non_sequences=[expA_flat]
)
# deflatten result
g_x = T.reshape(g_x_flat, newshape=expA.shape)

# here is where the computational graph is actually compiled
gradient = theano.function(inputs=[x], outputs=g_x)


# compute and display gradient computed with AD
display(gradient(2.))

array([[  8.57121973,   4.72067853,   8.04580421,   2.13757128],
       [ 18.6612217 ,  16.40946115,  21.30658595,   9.53122445],
       [  5.28795884,   5.08425043,   6.09091051,   2.89564534],
       [ 15.03470264,  15.63105922,  18.65004717,   9.20063013]])

array([[  8.56549989,   4.71673067,   8.0400172 ,   2.1355651 ],
       [ 18.64621668,  16.39642035,  21.28964497,   9.52387624],
       [  5.28352969,   5.08035571,   6.08585347,   2.89338313],
       [ 15.02161972,  15.61897565,  18.63482564,   9.19362008]])